https://www.kaggle.com/datasets/ethon0426/lending-club-20072020q1?select=Loan_status_2007-2020Q3.gzip

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
# import kagglehub
# ethon0426_lending_club_20072020q1_path = kagglehub.dataset_download('ethon0426/lending-club-20072020q1')

# print('Data source import complete.')


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [4]:
#df = pd.read_csv(ethon0426_lending_club_20072020q1_path+'/Loan_status_2007-2020Q3.gzip', low)

In [5]:
df=pd.read_csv("Loan_status_2007-2020Q3.csv", low_memory=False)

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
cleaning = df.drop_duplicates()

In [ ]:
nan_by_col = cleaning.isna().sum().reset_index()
nan_by_col.columns = ["columns", "missing"]
nan_by_col = nan_by_col.set_index("columns")
print(nan_by_col)

In [ ]:
nan_by_col["valid"] = [len(cleaning) - c for c in nan_by_col["missing"]]
nan_by_col["missing%"] = (cleaning.isna().sum()/cleaning.shape[0])*100
print(nan_by_col)

In [ ]:
g = nan_by_col[["missing", "valid"]].plot(kind = "barh", stacked= "true")
g.set_xlabel("Nr.rows")
g.set_title("Number of missing values by column")

In [ ]:
num_of_cells_with_nan = cleaning.isna().sum().sum()
print(f"Cells with NaN: {num_of_cells_with_nan}")
min_one_nan = cleaning.isna().any(axis = 1).sum()
print(f"Rows with at least one NaN: {min_one_nan}")
all_nan = cleaning.isna().all(axis = 1).sum()
print(f"Rows with all NaN: {all_nan}")

In [ ]:
percent_missing_per_row = (cleaning.isna().sum(axis = 1) / len(cleaning.columns))*100
percent_missing_per_row.hist(figsize = (5, 3))

In [ ]:
columns_to_drop = cleaning.columns[cleaning.notna().sum() < 282076]
print(columns_to_drop)

In [ ]:
cleaning.dropna(axis=1, thresh=2910000 , inplace=True) # Remove columns with less than 10 non-missing values

In [ ]:
cleaning.info()

In [ ]:
nan_by_col = cleaning.isna().sum().reset_index()
nan_by_col.columns = ["columns", "missing"]
nan_by_col = nan_by_col.set_index("columns")
print(nan_by_col)

In [ ]:
cleaning.head()

In [ ]:
for col in cleaning.columns:
    
    print(cleaning[col].value_counts())

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iter_imputer = IterativeImputer()
iter_data = cleaning.copy()

# Select only numerical features for imputation
numerical_columns = iter_data.select_dtypes(include=['number']).columns
numerical_data = iter_data[numerical_columns]

# Perform imputation on numerical data only
iter_data[numerical_columns] = iter_imputer.fit_transform(numerical_data)

#iter_data = pd.DataFrame(iter_data, columns=cleaning.columns, index=cleaning.index) # No need to recreate DataFrame

In [ ]:
# #should I still use KNN imputer with so many outliers ? Should I remove the outliers first?
# from sklearn.impute import KNNImputer

# knn_imputer = KNNImputer(n_neighbors = 5)
# knn_data = cleaning.copy()
# numerical_columns = knn_data.select_dtypes(include=['number']).columns
# numerical_data = knn_data[numerical_columns]
# knn_data[numerical_columns] = knn_imputer.fit_transform(numerical_data)


In [ ]:
# # two histplot to compare the imputed data (knn_data and iter_data)  on top of each other for features "MonthlyIncome","NumberOfDependents"

# fig, ax = plt.subplots(nrows= 2, figsize=(10, 10))
# sns.histplot(knn_data["MonthlyIncome"], ax=ax[0], kde=True, label = 'knn')
# sns.histplot(iter_data["MonthlyIncome"], ax=ax[0], kde=True, label = 'iterative')
# ax[0].set_title("monthly income")
# ax[0].legend()
# ax[0].set_ylim(0, 15000)
# ax[0].set_xlim(0, 20000)
# sns.histplot(knn_data["NumberOfDependents"], ax=ax[1], kde=True, label = 'knn')
# sns.histplot(iter_data["NumberOfDependents"], ax=ax[1], kde=True, label = 'iterative')
# ax[1].set_title("NumberOfDependents")
# ax[1].legend()

# plt.show();


In [ ]:
nan_by_col["valid"] = [len(iter_data) - c for c in nan_by_col["missing"]]
nan_by_col["missing%"] = (iter_data.isna().sum()/cleaning.shape[0])*100
print(nan_by_col)

In [ ]:
iter_data.dropna(how='any', inplace=True)

In [ ]:
nan_by_col["valid"] = [len(iter_data) - c for c in nan_by_col["missing"]]
nan_by_col["missing%"] = (iter_data.isna().sum()/cleaning.shape[0])*100
print(nan_by_col)


In [ ]:
Q1 = iter_data[numerical_columns].quantile(0.25)
Q3 = iter_data[numerical_columns].quantile(0.75)
IQR = Q3 - Q1
outlier_condition = ((iter_data[numerical_columns] < (Q1 - 1.5 * IQR)) | (iter_data[numerical_columns] > (Q3 + 1.5 * IQR))).any(axis=1)
outlier_indices = iter_data[outlier_condition].index 
iter_data.drop(outlier_indices, inplace=True)

In [ ]:
iter_data.info()

In [ ]:
iter_data['loan_status'].value_counts()

In [ ]:
clean_data = iter_data.copy()
#if loan_status = current or issued, drop the row
clean_data = clean_data[~clean_data['loan_status'].isin(['Current', 'Issued'])]
clean_data['loan_status'] = iter_data['loan_status'].apply(lambda x: 0 if x in ['Fully Paid'] else 1)

In [ ]:
clean_data.drop(columns=['id', 'url', 'pymnt_plan','Unnamed: 0'], inplace = True)

In [ ]:
df_object = clean_data.select_dtypes(include=['object'])
df_object.head()

In [ ]:
for col in df_object.columns :
        print(clean_data[col].value_counts())


In [ ]:
#clean_data['term'] = clean_data['term'].str.split(' ').str[0].astype(int)
clean_data['term'] = clean_data['term'].str.extract(r'(\d+)').astype(int)

In [ ]:
clean_data['int_rate'] = clean_data['int_rate'].str.rstrip('%').astype(float) / 100
clean_data['revol_util'] = iter_data['revol_util'].str.rstrip('%').astype(float) / 100


In [ ]:
clean_data['revol_util'] = iter_data['revol_util'].str.rstrip('%').astype(float) / 100


In [ ]:
clean_data['initial_list_status'] = np.where(clean_data['initial_list_status'] == 'w', 1, 0)
clean_data['debt_settlement_flag'] = np.where(clean_data['debt_settlement_flag'] == 'Y', 1, 0)


In [ ]:
date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
for col in date_columns:
    clean_data[col] = pd.to_datetime(clean_data[col], format='%b-%Y')  # Adjust format if needed
clean_data['time_since_earliest_cr_line'] = (clean_data['issue_d'] - clean_data['earliest_cr_line']).dt.days
clean_data['time_since_last_pymnt'] = (clean_data['issue_d'] - clean_data['last_pymnt_d']).dt.days
clean_data['days_since_last_credit_pull'] = (pd.Timestamp.now() - clean_data['last_credit_pull_d']).dt.days
clean_data = clean_data.drop(columns=date_columns)

In [ ]:
#clean_data.drop(columns=['pymnt_plan', 'Unnamed: 0'], inplace=True)

In [ ]:
clean_data.drop(columns=['zip_code'], inplace=True)

In [ ]:
clean_data.head()

In [ ]:
Y = clean_data['loan_status'].astype(int)

In [ ]:
Y.head()

In [ ]:
clean_data.info()

In [ ]:
object_cols = clean_data.select_dtypes(include=['object']).columns.tolist()  
# Strip whitespace and replace empty strings with NaN
clean_data[object_cols] = clean_data[object_cols].apply(lambda x: x.str.strip().replace('', np.nan))

# Now check for nulls again
print(clean_data[object_cols].isnull().sum())

In [ ]:
encoded2 = pd.get_dummies(clean_data.drop(columns=['loan_status']), 
                       columns=object_cols, 
                       dummy_na=True)
print(np.isnan(encoded2).sum())

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoded = clean_data.copy()
encoded.drop(columns=['loan_status'], inplace= True)
# encoded = encoded.drop(columns=['loan_status'])
# # Assuming 'df' is your DataFrame
# 

# # Create a OneHotEncoder object
# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  

# # Fit the encoder to your categorical features
# encoded_data = encoder.fit_transform(encoded[object_cols])

# # Create a DataFrame from the encoded data
# encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_cols))

# # Concatenate the encoded DataFrame with the original DataFrame
# encoded = pd.concat([encoded, encoded_df], axis=1)

# # Drop the original categorical features 
# encoded = encoded.drop(columns=object_cols)
# Ensure all categorical columns are strings
encoded[object_cols] = encoded[object_cols].astype(str)

# Reinitialize encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(encoded[object_cols])

# Ensure index alignment
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_cols), index=encoded.index)

# Drop original columns and concatenate
encoded = pd.concat([encoded.drop(columns=object_cols), encoded_df], axis=1)

In [ ]:
encoded.info()

In [ ]:
encoded.isna().sum()

In [ ]:
#encoded.to_csv('my_dataframe.csv', index=False)
p2p_loan_data = pd.concat([Y,encoded], axis=1)
p2p_loan_data.to_csv('p2p_loan_data.csv', index=False)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Remove features with <1% variance
selector = VarianceThreshold(threshold=0.01)
X_filtered = selector.fit_transform(encoded)
var_filtered = pd.DataFrame(X_filtered, columns = selector.get_feature_names_out())
# Assuming 'df' is your DataFrame
variances = var_filtered.var().sort_values(ascending=False)
plt.figure(figsize=(10, 60))  # Adjust figure size as needed
plt.barh(variances.index, variances.values)
plt.title('Feature Variances')
plt.xlabel('Variance')
plt.ylabel('Features')
plt.show()

In [ ]:


# Drop highly correlated numerical features (Pearson |corr| > 0.9)
corr_matrix = var_filtered.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
corr_filtered = var_filtered.drop(to_drop, axis=1)

In [ ]:
correlation_matrix = corr_filtered.corr()
plt.figure(figsize=(10, 8))  # Adjust figure size as needed
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.savefig('Correlation Matrix.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2,mutual_info_classif

# Select top 20 numerical features
selector = SelectKBest(mutual_info_classif, k=20)
X_selected = selector.fit_transform(corr_filtered, Y)  # X_num = numerical subset


In [ ]:
selector.get_feature_names_out()

In [ ]:
scores = selector.scores_
feature_names = corr_filtered.columns

# Create a DataFrame for easy sorting
feature_scores = pd.DataFrame({'Feature': feature_names, 'Score': scores})
feature_scores = feature_scores.sort_values('Score', ascending=False)

# Plot (horizontal bar chart)
plt.figure(figsize=(10, 8))
plt.barh(feature_scores['Feature'][:20], feature_scores['Score'][:20], color='skyblue')
plt.xlabel('Importance Score (Mutual Information)')
plt.title('Top 20 Features by Mutual Information Score')
plt.gca().invert_yaxis()  # Highest score at top
plt.savefig('Top 20 Features by MI filtered.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
selector = SelectKBest(mutual_info_classif, k=20)
X_selected = selector.fit_transform(encoded, Y) 
selector.get_feature_names_out()

In [ ]:
scores = selector.scores_
feature_names = encoded.columns

# Create a DataFrame for easy sorting
feature_scores = pd.DataFrame({'Feature': feature_names, 'Score': scores})
feature_scores = feature_scores.sort_values('Score', ascending=False)

# Plot (horizontal bar chart)
plt.figure(figsize=(10, 8))
plt.barh(feature_scores['Feature'][:20], feature_scores['Score'][:20], color='skyblue')
plt.xlabel('Importance Score (Mutual Information)')
plt.title('Top 20 Features by Mutual Information Score')
plt.gca().invert_yaxis()  # Highest score at top
plt.savefig('Top 20 Features by MI encoded.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
X_selected = pd.DataFrame(X_selected, columns= selector.get_feature_names_out())
correlation_matrix = X_selected.corr()
plt.figure(figsize=(10, 8))  # Adjust figure size as needed
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.savefig('Correlation Matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
# model.fit(corr_filtered, Y)
# selected_features = corr_filtered.columns[model.coef_[0] != 0]

    

In [ ]:
# for feature in selected_features:
#     i = 0
#     if model.coef_[i] != 0
#         print(feature, model.coef_[i])
#     i +=1

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
rfe = RFECV(model, min_features_to_select=10, cv=5)
rfe.fit(encoded, Y)
selected_features = encoded.columns[rfe.support_]

In [ ]:
print(selected_features)

In [ ]:
# Plot CV scores vs. number of features
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(rfe.cv_results_['mean_test_score']) + 1),
         rfe.cv_results_['mean_test_score'],
         marker='o',
         linestyle='--',
         color='b')

# Highlight the optimal number of features
optimal_features = rfe.n_features_
plt.axvline(x=optimal_features, color='r', linestyle=':',
            label=f'Optimal: {optimal_features} features')

plt.xlabel('Number of Features Selected')
plt.ylabel('Mean CV Score (Accuracy)')
plt.title('RFECV: Optimal Number of Features')
plt.legend()
plt.grid(True)
plt.savefig('RFECV_scores.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create a DataFrame of feature rankings
rankings = pd.DataFrame({
    'Feature': encoded.columns,
    'Rank': rfe.ranking_,
    'Selected': rfe.support_
}).sort_values('Rank')

# Plot (horizontal bar chart)
plt.figure(figsize=(10, 8))
colors = np.where(rankings['Selected'], 'green', 'red')  # Green=selected, Red=dropped
plt.barh(rankings['Feature'], rankings['Rank'], color=colors)
plt.xlabel('Feature Rank (1=Selected)')
plt.title('RFECV Feature Rankings')
plt.gca().invert_yaxis()  # Highest rank (1) at top
plt.grid(axis='x')

# Add legend
import matplotlib.patches as mpatches
green_patch = mpatches.Patch(color='green', label='Selected')
red_patch = mpatches.Patch(color='red', label='Dropped')
plt.legend(handles=[green_patch, red_patch])
plt.savefig('RFECV_rankings.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Plot 1: CV Scores
ax1.plot(range(1, len(rfe.cv_results_['mean_test_score']) + 1), 
         rfe.cv_results_['mean_test_score'], 
         marker='o', color='b')
ax1.axvline(optimal_features, color='r', linestyle=':', 
            label=f'Optimal: {optimal_features} features')
ax1.set_title('Optimal Feature Count', fontsize=14)
ax1.set_xlabel('Number of Features')
ax1.set_ylabel('CV Score')
ax1.legend()

# Plot 2: Feature Rankings
colors = np.where(rankings['Selected'], '#1f77b4', '#ff7f0e')  # Blue=selected, Orange=dropped
ax2.barh(rankings['Feature'], rankings['Rank'], color=colors)
ax2.set_title('Feature Selection Rankings', fontsize=14)
ax2.set_xlabel('Rank (1=Selected)')
ax2.invert_yaxis()

plt.tight_layout()
plt.savefig('RFECV_Results.png', dpi=300, bbox_inches='tight')  # Save for slides
plt.show()
